In [3]:
import subprocess
import pandas as pd

In [14]:
model_name = 'models/30jun_4modesB2'
model_data = 'models-data/data_30junB.dat'

In [18]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biogeme.exe', model_name,model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]\r\n'
b'Michel Bierlaire, EPFL\r\n'
b'-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1\r\n'
b'See http://biogeme.epfl.ch\r\n'
b'                    !! CFSQP is available !!\r\n'
b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'\t"In every non-trivial program there is at least one bug."\r\n'
b'\r\n'
b'\r\n'
b'[01:11:44]patFileNames.cc:52  models/30jun_4modesB2.par does not exist\r\n'
b'[01:11:44]patFileNames.cc:56  Trying default.par instead\r\n'
b'[01:11:44]patBiogeme.cc:134  Read default.par\r\n'
b' Opening file models-data/data_30junB.dat\r\n'
b' Data  file... line 500\tMemory: 59 Kb\r\n'
b' Total obs.:   837\r\n'
b' Total memory: 99.8398 Kb\r\n'
b' Run time for data processing: 00:00\r\n'
b' Init loglike=-985.941\r\n'
b'     gmax Iter   radius        f(x)     Status       rhok nFree\r\n'
b' +1.01e+000    1 1.00e+000 +9.8594093e+002 ****OutTrReg +1.00e+000 5  

In [19]:
with open(model_name+'_res.mod','wb') as simmod:
    with open(model_name+'.res','rb') as res:
        searchlines = res.readlines()
        foundit = False
        for i, line in enumerate(searchlines):
            if foundit==False:
                if b'[SampleEnum]' in line:
                    print('Found it.')
                    foundit = True
                simmod.write(line)
            else:
                if line.decode("utf-8").find('0\r\n')==0:
                    print('Changed to 100.')
                    simmod.write(b'100\r\n')
                    foundit==False
                else:
                    simmod.write(line)

Found it.
Changed to 100.


In [20]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biosim.exe', model_name+'_res',model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]\r\n'
b'Michel Bierlaire, EPFL\r\n'
b'-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1\r\n'
b'See http://biogeme.epfl.ch\r\n'
b'                    !! CFSQP is available !!\r\n'
b'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n'
b'\t"In every non-trivial program there is at least one bug."\r\n'
b'\r\n'
b'\r\n'
b'[01:12:21]patFileNames.cc:52  models/30jun_4modesB2_res.par does not exist\r\n'
b'[01:12:21]patFileNames.cc:56  Trying default.par instead\r\n'
b'[01:12:21]patBiogeme.cc:134  Read default.par\r\n'
b'Warning:  Lower bound on mu set to 1e-005\r\n'
b'Warning:   Value defined by gevMinimumMu in default.par\r\n'
b' Opening file models-data/data_30junB.dat\r\n'
b' Data  file... line 500\tMemory: 59 Kb\r\n'
b' Total obs.:   837\r\n'
b' Total memory: 99.8398 Kb\r\n'
b' 59%\t[500/838] Est. term. time: 07/01/16 01:12:21 (in 00:00)\r\n'
b' BIOSIM Input file\r\n'
b' ===

In [45]:
simres = pd.read_csv(model_name+'_res.enu',sep='\t')
simres.head()

,Choice_Id,P_choice,V_A1_TAXI,V_A2_TRANSIT,V_A3_OTHER,V_A4_FREE,P_A1_TAXI,Residual_A1_TAXI,P_A2_TRANSIT,Residual_A2_TRANSIT,...,Simul91,Simul92,Simul93,Simul94,Simul95,Simul96,Simul97,Simul98,Simul99,Simul100
0,1,0.561875,0.192995,-0.741756,-0.756148,-9999.99,0.561875,0.438125,0.220639,-0.220639,...,2,2,1,3,1,1,2,1,2,1
1,1,0.561875,0.192995,-0.741756,-0.756148,-9999.99,0.561875,0.438125,0.220639,-0.220639,...,1,1,1,1,1,1,1,1,3,3
2,1,0.561875,0.192995,-0.741756,-0.756148,-9999.99,0.561875,0.438125,0.220639,-0.220639,...,2,2,1,2,1,2,3,1,3,1
3,1,0.561875,0.192995,-0.741756,-0.756148,-9999.99,0.561875,0.438125,0.220639,-0.220639,...,1,1,2,1,2,1,3,2,2,3
4,1,0.561875,0.192995,-0.741756,-0.756148,-9999.99,0.561875,0.438125,0.220639,-0.220639,...,2,2,1,2,1,1,1,1,2,1


In [46]:
simul_start = list(simres.columns).index('Simul1')
from collections import Counter
def count_correct(row):
    return list(row[simul_start:]).count(row.Choice_Id)
def simul_mode(row):
    return int(Counter(list(row[simul_start:])).most_common(1)[0][0])

In [47]:
simres['Accuracy'] = simres.apply(count_correct,axis=1)
simres['Mode_Prediction'] = simres.apply(simul_mode,axis=1)

In [50]:
simres['ModeAccurate'] = simres['Choice_Id']==simres['Mode_Prediction']

In [38]:
simres.groupby('Choice_Id')['Accuracy'].mean()

Choice_Id
1    57.871429
2    21.865546
3    23.070423
4    99.991189
Name: Accuracy, dtype: float64

In [51]:
simres.groupby('Choice_Id')['ModeAccurate'].mean()

Choice_Id
1    0.982857
2    0.084034
3    0.000000
4    1.000000
Name: ModeAccurate, dtype: float64